In [1]:
import pyvista as pv
import numpy as np
import math  
import trame
import networkx as nx
import time


from tqdm import tqdm
from Voxel_model import Voxels
from Path_create import Create_Path
from creation_vector import Vector_Creation
from mesh_tools import centrer_0xy , rotation_mesh , midle_xy ,translation

from Free_path import Free_path
from Slicing import collision , collision_set_layer , collision_intersect_layer

# import trimesh
# import pyvox
# import pyvoxsurf

# Test Path

In [2]:
path = Create_Path('Object//UM3E_Copy of Cube Extruder (3).gcode')
path.read_gcode(12)

Collision = Create_Path('Object//UM3E_collision.gcode')
Collision.read_gcode(11)

path.centrer(Collision.tool_path)

mesh_collision = centrer_0xy('Object/collision.stl')

path.figure(mesh_collision)

Widget(value="<iframe src='http://localhost:60840/index.html?ui=P_0x10640f9d0_0&reconnect=auto' style='width: …

# From STL to Voxel

In [3]:
scale_voxel = 0.5
scale_print= 500
vox = Voxels(scale_print,scale_voxel)

In [4]:
voxels_collision = pv.voxelize(mesh_collision, density=scale_voxel, check_surface=False)
voxels_collision.points = np.divide(voxels_collision.points,scale_voxel).astype(int)
mesh_collision.points = np.divide(mesh_collision.points,scale_voxel).astype(int)

In [5]:
centre = midle_xy(voxels_collision)
voxels_collision.points = translation(voxels_collision.points,centre,vox.midle)
mesh_collision.points = translation(mesh_collision.points,centre,vox.midle)

path.correction(vox.scale_voxel)
path.translat(centre,vox.midle)
path.figure(mesh_collision)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyvista/utilities/helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value="<iframe src='http://localhost:60840/index.html?ui=P_0x162849750_1&reconnect=auto' style='width: …

In [6]:
mesh_head_print=  pv.read('Object/Deposition Head.stl')
debut = time.perf_counter()
voxels_head_print = pv.voxelize(mesh_head_print, density=scale_voxel, check_surface=False)
fin = time.perf_counter()
print(fin-debut)
voxels_head_print.points = np.divide(voxels_head_print.points,vox.scale_voxel).astype(int)
mesh_head_print.points = np.divide(mesh_head_print.points,vox.scale_voxel).astype(int)

29.258944208035246


In [7]:
voxels_collision_surface = voxels_collision.extract_surface()
voxels_head_print = voxels_head_print.extract_surface()
voxels_head_print.points[:,2]+=6

In [8]:
vox.add_density(voxels_collision_surface.points)

 JS Error => error: TypeError: Invalid mix of BigInt and other type in multiplication.


# Tool Vectors

In [ ]:
theta , theta_max = 10 , 50
Circle=Vector_Creation(np.deg2rad(theta),np.deg2rad(theta_max))

position_head_layer_circle = {i: [] for i in Circle }
G = nx.DiGraph()
tool_path = path.tool_path[5000:5200]
num_layers = len(tool_path)
layer_vertex = {i: [] for i in range(num_layers)}

# Free Path generation

- Free_path_parallelize_ThreadPool

In [ ]:
debut = time.perf_counter()
free_path= Free_path_parallelize_ThreadPool(vox, tool_path, voxels_head_print, theta , position_head_layer_circle,Circle, G, layer_vertex )
fin = time.perf_counter()
print(fin-debut)

- Free_path_parallelize_ProcessPool

In [ ]:
debut = time.perf_counter()
free_path= Free_path_parallelize_ProcessPool(vox, tool_path, voxels_head_print, theta , position_head_layer_circle,Circle, G, layer_vertex )
fin = time.perf_counter()
print(fin-debut)

- Free_path

In [ ]:
debut = time.perf_counter()
free_path= Free_path(vox, tool_path, voxels_head_print, theta , position_head_layer_circle,Circle, G, layer_vertex )
fin = time.perf_counter()
print(fin-debut)

# See results

In [ ]:
free_path=free_path[1:-1]

In [ ]:
p=pv.Plotter()
ele=free_path[0]
position_rotate = position_head_layer_circle[ele[1]][ele[2]].copy()
position_rotate = translation(position_rotate,tool_path[-1],tool_path[ele[0]])

p.add_mesh(position_rotate, show_edges=False, color='blue')
p.add_mesh(mesh_collision, color=True, show_edges=True, opacity=0.7)
p.show()

# With Slicing

In [13]:
theta , theta_max = 10 , 50
Circle=Vector_Creation(np.deg2rad(theta),np.deg2rad(theta_max))

position_head_layer_circle = {i: [] for i in Circle }
G = nx.DiGraph()
tool_path = path.tool_path[5000:5001]
num_layers = len(tool_path)
layer_vertex = {i: [] for i in range(num_layers)}

In [ ]:
tool_path

In [14]:
# collision_set_layer or collision_intersect_layer
                    
debut = time.perf_counter()
free_path= Free_path_Slicing(tool_path, voxels_head_print, voxels_collision_surface , theta , position_head_layer_circle,Circle, G, layer_vertex, collision_intersect_layer)
fin = time.perf_counter()
print(fin-debut)

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]

1.497209541965276


 JS Error => error: TypeError: Invalid mix of BigInt and other type in addition.


# Create video

In [ ]:
import os
from video import create_images , create_video

In [ ]:
free_path_images = create_images(free_path,position_head_layer_circle,tool_path,mesh_collision)
os.environ['IMAGEIO_FFMPEG_EXE'] = '/opt/homebrew/Cellar/ffmpeg/6.0/bin/ffmpeg'
create_video(free_path_images, "Object/animation.mp4")

# Parralelize code 

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
def Free_path_parallelize_ThreadPool(vox, tool_path, surface, theta, position_head_layer_circle, Circle, G, layer_vertex):
    z_max_collision = voxels_collision.points[:,2].max()
    num_layers = len(tool_path)
    source = 's'
    target = 't'

    G.add_node(source)
    G.add_node(target)
        
            
    with ThreadPoolExecutor(8) as executor:
        for i in tqdm(range(num_layers)):
            if i > 0:
                prev_layer = i - 1
                prev_layer_vertices = layer_vertex[prev_layer]
            else :
                    prev_layer_vertices = source
            iterations_args = [(delta, ind, j,G,theta,tool_path, position_head_layer_circle, layer_vertex, prev_layer_vertices, vox, surface, i, target, source , z_max_collision) for j in Circle for ind, delta in enumerate(np.rad2deg(Circle[j]))]

                # Paralléliser les appels à la fonction process_iteration
            executor.map(process_iteration, iterations_args)

            vox.voxels[tool_path[i][0], tool_path[i][1], tool_path[i][2]] = 1

    shortest_path = nx.dijkstra_path(G, source, target)
    return  shortest_path

In [ ]:
def process_iteration(args):
    delta, ind, j,G,theta,tool_path, position_head_layer_circle, layer_vertex, prev_layer_vertices, vox, surface, i, target, source , z_max_collision= args
    
    
    if i == 0:
        position_rotate = rotation_mesh(surface, delta, j * theta)
        position_rotate = np.floor(position_rotate.points).astype(int)
        position_head_layer_circle[j].append(position_rotate)
        position = translation(position_rotate, np.array([0, 0, 0]), tool_path[i])
    else:
        position_rotate = position_head_layer_circle[j][ind]
        position = translation(position_rotate, tool_path[i - 1], tool_path[i])
        
    mask = position[:,2]<= z_max_collision
        
    if not vox.density(position[mask]):  # s'il n'y a pas de desnité (pas d'obstacle)
        
        current_vertex = (i, j, ind)  # name of the vertex
        G.add_node(current_vertex, angle_z=j * theta)  # creer le sommet white atribute
        layer_vertex[i].append(current_vertex)  # Ajoute les sommets aux listes d'étages
        if i > 0:
            for prev_vertex in prev_layer_vertices:  # creer des liaison entre les etages si la condition angulaire est respécté
                angle = abs(G.nodes[prev_vertex]['angle_z'] - G.nodes[current_vertex]['angle_z'])
                if angle <= theta:
                    G.add_edge(prev_vertex, current_vertex, weight=angle)
        if i == 0:
            G.add_edge(source, current_vertex, weight=0)
        if i == num_layers - 1:
            G.add_edge(current_vertex, target, weight=0)

# Slicing Code 

In [10]:
def Free_path_Slicing(tool_path, surface, voxels_collision_surface, theta, position_head_layer_circle,Circle,G, layer_vertex , function):
        num_layers = len(tool_path)
        source = 's'
        target = 't'
        
        G.add_node(source)
        G.add_node(target)
        
        for i in tqdm(range(num_layers)):
            if i > 0:
                prev_layer = i - 1    # Récupérez les sommets de l'étage précédent
                prev_layer_vertices = layer_vertex[prev_layer]

            for j in Circle:
                for ind, delta in enumerate(np.rad2deg(Circle[j])):
                    
                    if i == 0 :
                        position_rotate = rotation_mesh(surface, delta,j*theta) 
                        position_rotate = np.floor(position_rotate.points).astype(int)
                        position_head_layer_circle[j].append(position_rotate)
                        position = translation(position_rotate,np.array([0,0,0]),tool_path[i])
                    else :
                        position_rotate = position_head_layer_circle[j][ind]
                        position = translation(position_rotate,tool_path[i-1],tool_path[i])
                        
                    Coll = collision(position , voxels_collision_surface.points , function ) # collision_set_layer or collision_intersect_layer
                   
                    if not Coll: # s'il n'y a pas de desnité (pas d'obstacle)
                        current_vertex = (i, j , ind)  #  name of the vertex
                        G.add_node(current_vertex, angle_z=j*theta) # creer le sommet white atribute 
                        layer_vertex[i].append(current_vertex)  # Ajoute les sommets aux listes d'étages
                        if i > 0:
                            for prev_vertex in prev_layer_vertices: # creer des liaison entre les etages si la condition angulaire est respécté
                                angle = abs(G.nodes[prev_vertex]['angle_z']-G.nodes[current_vertex]['angle_z'])
                                if angle <= theta:
                                    G.add_edge(prev_vertex, current_vertex, weight=angle)
                        if i == 0:
                            G.add_edge(source, current_vertex, weight=0)
                        if i == num_layers-1:
                            G.add_edge(current_vertex,target ,weight=0)
                            
            vox.voxels[tool_path[i][0],tool_path[i][1],tool_path[i][2]]=1


       
        shortest_path = nx.dijkstra_path(G, source, target)
        
        return shortest_path 